# Test **tcheasy** with `python type hinting`
This notebook shows some examples for the usage of tcheasy utilizing the python hinting. <br>
As already mentioned on the landing page, not all types are supported.

Currently the following types are supported:
- **NoneType(s)**: None
- **number types**: int, float, complex, bool
- **sequence types**: str, list, tuple
- **mapping types**: dict
- **set types**: sets
- **callable types**: object
- **`typing`-module types**: typing.Any, typing.Union

Maybe there will be new types added in the future. <br>
<br>
Anyway. <br>
Another thing to keep in mind is the fact, that you are only able to use **tcheasy** and hints with the positional parameters. You are able to hint \*args & \**kwargs, but **tcheasy** flat out ignores it. <br>
The reason for this is that **tcheasy** would not be able to guess the upcoming (e.g.) \**kwargs variables by the hint of:<br>

<br>

```python
def kwargs_hint(**kwargs:int)
```

<br>

This type of definition would be to restrictive. <br>

So let's jump in and do some type checking!

In [1]:
# import the tcheasy decorator
from tcheasy import tcheasy

In [2]:
# add the decorator to a function
@tcheasy()
def our_first_test(var1:str, var2:bool) -> bool:
    """Simple function which just returns True """
    
    return True

In [3]:
# now lets call the function
our_first_test("some-string",False)

True

In [4]:
# and now lets try to switch the types
our_first_test(False,1)

{'success': False,
 'error': "[K.2]: The parameter 'var1' needs to be a(n) str."}

Boom! <br>
As expected! <br>
When we pass the correct types, the function runs and returns its normal return value. But if we mess up the correct types, the function returns a dict with additional info for the user. <br>
<br>
Normaly you catch those error messages and convert them into the ones you like and need for your app. <br>
<br>
Each error message comes also with an error code. In the example from above, we received the `[K.2]` code, which means 'wrong type'. <br>
<br>
A full list is provided within the function help.

In [5]:
# get help!
help(tcheasy)

Help on function tcheasy in module tcheasy.type_check:

tcheasy(to_check: dict = {}, debug: bool = False) -> Any
    Type checks user input
    
    This function checks all passed types by
    utilizing either a 'to_check' dict
    or the python type hintings.
    
    NOTE:
    Prior checking, the algorithmn sorts all
    passed parameters into their correct 
    parameter type.
    
    There are three different python parameter
    types considered (also used for the 'to_check'
    definition; see below).
        positional: default, keyword, positional
            These parameters are your defined 
            parameters.
                Example: def example(a, b = 2)
        args: arbitrary positional
            These parameters are basically *args.
        kwargs: arbitrary keyword
            These parameters are basically
            **kwargs.
    
    The format of 'to_check' is
    ------------------------------------------
    
        {
            'positional':{
        

<br>
<br>

## Hints for \*args & \**kwargs
As already mentioned, **tcheasy** will ignore any type hints for `*args` and `**kwargs`:

In [6]:
# function with args and kwargs hints
@tcheasy()
def with_arbitrary_hints(var1:str, var2:bool, *args:int, **kwargs:float) -> bool:
    """Simple function which just returns True """
    
    return True

In [7]:
# test the function --> *args & **kwargs (5 & z) should trigger
with_arbitrary_hints("string", False, 5, z="another-string")

True

... while when we call the same function and mess up the `var1` & `var2`:

In [8]:
# messing around with var1 & var2
with_arbitrary_hints(5, True, 5, z=6)

{'success': False,
 'error': "[K.2]: The parameter 'var1' needs to be a(n) str."}

<br>
<br>

## Hints with default values

**tcheasy** also works with default values:

In [9]:
# a example function with defaults, this time it returns the params
@tcheasy()
def with_defaults(a:bool = True, b = 2):
    """Simple function which just returns the passed params """
    return {'a':a, 'b':b}

In [10]:
# call the function without any arguments
with_defaults()

{'a': True, 'b': 2}

As you can see, the defaults were passed automatically. <br>
Now we are going to pass `b`:

In [11]:
# now only pass b
with_defaults(b=15)

{'a': True, 'b': 15}

In [12]:
# pass wrong type for a
with_defaults(a=13, b=True)

{'success': False, 'error': "[K.2]: The parameter 'a' needs to be a(n) bool."}

<br>
<br>

## Mixed hinting
As shown previously, you can mix parameters with and without hinting. In these cases **tcheasy** applies for the 'unhinted' parameters any default supported type (see the list at the top). <br>
<br>
To show you what this, lets build a function and activate the debug printings for **tcheasy**:

In [13]:
# function with debug prints
@tcheasy(debug=True)
def debug(a, b:int):
    """Function which returns True"""
    return True

In [14]:
# call the function and see what happens
result = debug(15,15)

SORTED:  {'positional': {'a': 15, 'b': 15}, 'args': [], 'kwargs': {}, 'hinting': {'b': <class 'int'>, 'a': (<class 'NoneType'>, <class 'int'>, <class 'float'>, <class 'complex'>, <class 'bool'>, <class 'str'>, <class 'list'>, <class 'tuple'>, <class 'dict'>, <class 'set'>, <class 'object'>)}, 'declared': ['a', 'b']}
FLAGS:  {'positional': True, 'args': False, 'kwargs': False}
MOD: {'positional': {'b': 15, 'a': 15}, 'args': [], 'kwargs': {}, 'hinting': {'b': <class 'int'>, 'a': (<class 'NoneType'>, <class 'int'>, <class 'float'>, <class 'complex'>, <class 'bool'>, <class 'str'>, <class 'list'>, <class 'tuple'>, <class 'dict'>, <class 'set'>, <class 'object'>)}, 'declared': ['a', 'b']}
MOD ARGS: (15, 15)
MOD KWARGS: {}


Wooooh... that's a lot of info, but just bare with me for a second. <br>
<br>
When you inspect the `SORTED` more closely, you can see that there is the keyword `hinting`. <br>
This is actually what **tcheasy** sees when it inspects your function at runtime. <br>
Within this dict, you can see only one type for `b`, but a lot for `a`. To be specific: basically every supported type. You can check it if you don't believe me ;-)

<br>
<br>

## Add 'or' hints

You are also able to add multiple types (or `Any`) for a positional parameter. <br>
To do this, you have to import python's `typing` module:

In [15]:
# import typing
import typing

From there on you can use the `Union` to specify multiple possible types. <br>
Keep in mind that **tcheasy** chan only work with the supported types though!

In [16]:
# function with union
@tcheasy()
def with_union(u:typing.Union[str,int], b:typing.Any) -> bool:
    """Function which returns True"""
    return True

In [17]:
# call it with correct types
with_union(u="hallo", b=None)

True

In [18]:
# again correct
with_union(u=1, b=True)

True

In [19]:
# now its the wrong type
with_union(u=.1, b=.1)

{'success': False,
 'error': "[K.2]: The parameter 'u' needs to be a(n) str | int."}

Did you notice that somehow no argument that we entered for `b` was considered as wrong? Thats because we did use the `Any` type which is actually every supported type (and thus pretty boring to use).

<br>
<br>

# It's a wrap!

That's it for now. In this little example you learned the basic usage of **tcheasy** and its restrictions when running it with the python type hinting module. <br>

<br>

<p style='color:gray'><small><em>PS: did you notice that none of the input checks did actually brick the jupyter notebook when running it all? Thats because <b>tcheasy</b> translates every exception into those dict returns :-) </em></small></p>